In [1]:
# Load packages
import pandas as pd
import requests
import json
from pathlib import Path

In [2]:
# Define file name
file_name = 'consumer-price-index_us_1982-84_2022'

In [3]:
# Define series id for API request
seriesid = 'CUUR0000SA0' # CPI for All Urban Consumers (CPI-U) 1982-84=100 (Unadjusted) - CUUR0000SA0

In [4]:
# Request series data from API
headers = {'Content-type': 'application/json'}
data = json.dumps({
    'seriesid': [seriesid],
    'startyear': 2022,
    'endyear': 2022
    })
r = requests.post('https://api.bls.gov/publicAPI/v1/timeseries/data/', headers=headers, data=data)

In [5]:
# Create data frame from json response
df = pd.DataFrame.from_dict(r.json()['Results']['series'][0])

In [6]:
# Split data column into individual columns
df = df.data.apply(pd.Series)

In [7]:
# Create monthly period index
df.period = df.year + df.period
df.period = df.period.str.replace('M', '-')
df.index = pd.DatetimeIndex(df.period).to_period('M')

In [8]:
# Drop all columns except value
df = df.loc[:, ['value']]

In [9]:
# Rename value column
df.columns = ['cpi']

In [10]:
# Convert CPI to numeric
df['cpi'] = pd.to_numeric(df['cpi'])

In [11]:
# Sort values by month
df.sort_index(inplace=True)

In [12]:
# Use most recent pandas data types (e.g. pd.NA)
df = df.convert_dtypes()

In [13]:
# Save CPI data
df.to_csv(f'../data/raw/{file_name}.csv')